# Twitch Data Collection

## Testing the API

In [1]:
# Import necessary libraries
import pandas as pd
import requests
import json

Getting secrets from .env file in root of project folder. Token generated from here https://dev.twitch.tv/docs/api

In [4]:
from dotenv import load_dotenv
import os 

# load environment
load_dotenv()

url = "https://api.twitch.tv/helix/games/top?first=40"
CLIENT_ID = os.environ.get('CLIENT_ID')
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')
ACCESS_TOKEN = os.environ.get('ACCESS_TOKEN')

headers = {
    'Client-ID': CLIENT_ID,
    'Authorization': 'Bearer ' + ACCESS_TOKEN
}

In [5]:
requests.get(url, headers=headers).json()

{'data': [{'id': '509658',
   'name': 'Just Chatting',
   'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/509658-{width}x{height}.jpg'},
  {'id': '21779',
   'name': 'League of Legends',
   'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/21779-{width}x{height}.jpg'},
  {'id': '263490',
   'name': 'Rust',
   'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/263490_IGDB-{width}x{height}.jpg'},
  {'id': '516575',
   'name': 'VALORANT',
   'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/516575-{width}x{height}.jpg'},
  {'id': '32982',
   'name': 'Grand Theft Auto V',
   'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/32982_IGDB-{width}x{height}.jpg'},
  {'id': '29595',
   'name': 'Dota 2',
   'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/29595-{width}x{height}.jpg'},
  {'id': '511224',
   'name': 'Apex Legends',
   'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/511224-{width}x{height}.jpg'},
  {'id': '33214',
   'name': 'Fortnite',
 

Generating a new token for each call to the API

In [6]:
AUTH_URL = "https://id.twitch.tv/oauth2/token"
CLIENT_ID = os.environ.get('CLIENT_ID')
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')
GRANT_TYPE = "client_credentials"

# post a request to get a token using my client id and client secret
params = {
  "client_id": CLIENT_ID,
  "client_secret": CLIENT_SECRET,
  "grant_type": GRANT_TYPE
}

r1 = requests.post(url=AUTH_URL, params=params)
token = r1.json()["access_token"]

In [7]:
# get request

URL = "https://api.twitch.tv/helix/search/channels?query=reisu1337"

headers = {
    'Client-ID': CLIENT_ID,
    'Authorization': 'Bearer ' + token
}

requests.get(URL, headers=headers).json()

{'data': [{'broadcaster_language': 'en',
   'broadcaster_login': 'reign1337',
   'display_name': 'Reign1337',
   'game_id': '509658',
   'game_name': 'Just Chatting',
   'id': '95707013',
   'is_live': False,
   'tag_ids': [],
   'thumbnail_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/reign1337-profile_image-77df4c4bcf4e390e-300x300.png',
   'title': 'quick workout',
   'started_at': ''},
  {'broadcaster_language': '',
   'broadcaster_login': 'resu13371',
   'display_name': 'resu13371',
   'game_id': '0',
   'game_name': '',
   'id': '169360499',
   'is_live': False,
   'tag_ids': [],
   'thumbnail_url': 'https://static-cdn.jtvnw.net/user-default-pictures-uv/de130ab0-def7-11e9-b668-784f43822e80-profile_image-300x300.png',
   'title': '',
   'started_at': ''},
  {'broadcaster_language': 'en',
   'broadcaster_login': 'reisu1337',
   'display_name': 'Reisu1337',
   'game_id': '27471',
   'game_name': 'Minecraft',
   'id': '576638480',
   'is_live': False,
   'tag_ids': [],
   'th

## Framing the project

In this section I will outline what I am aiming to achieve with this piece of work. In particular, I will describe:
- The questions that I hope to answer
- What data I will use to answer these questions

### Questions/Hypothesis

#1 There is a positive relationship between minutes streamed and followers

#2 Mature content is more popular

#3 What is the best time for a streamer to go live?

#4 What are the top games on Twitch?

#5 Who is the most popular streamer on Twitch in terms of followers/subscriptions/views?


### Data requirements

I will collect the data from the Twitch API

I will separate the data into two dataframes

1. Channels
- Channel Name
- Channel Start Date (?)
- Watch Time (minutes)
- Stream Time (minutes)
- Peak Viewers
- Average Viewers
- Followers
- Followers gained
- Views gained
- Subscription count
- Max subscription count (?)
- Partnered
- Mature

2. Games
- Game Name
- Month (?)
- Year (?)
- Rank in the month
- Hours watched
- Hours streamed on twitch
- Peak viewers
- Peak Channels
- Streamers
- Average viewers

## Channel dataset

### Get one channel

[Search channels](https://dev.twitch.tv/docs/api/reference#search-channels) gives us the channel ID and the channel name.
[Get Users](https://dev.twitch.tv/docs/api/reference#get-users) gives us Views gained and stream created date.